# putting together friends dataframe

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

import nltk
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk import wordnet
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import learning_curve
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve

In [2]:
Laurents = ["Haven't posted in a while, so this random picture is now filling the void. Been going within a lot in the past year and sorted out some things for myself. Happy to say, that I am refocused, re-energized and working on great things.", 
            "Impressions from Serfaus. 2020 is off to a good start. Swipe right for the best drinking game", 
         "Dogs of Berlin", 
         "Not my boat, still flexing ", 
         "Lads", 
         "A weekend in Madrid is always a good idea! Hanging out the hangover at some historical sites ", 
         "Drip too hard, don’t stand too close", 
         "I miss you bro. We pushed each other to be the best every day. There never was a better duo. „The Golden Boys“ Rest Easy",
        "Getting settled in Berlin",
        "It’s been three tough but fun years at WHU. Last weekend I finally graduated with a degree from the best business school in Germany, but more importantly with new friends for life and another place I can call home.",
         
        "Graduated",
        "WHU boys - Class of 2017. Had so much fun with you. Now let's get the moolah",
        "The pic is blurry but the day was sharp",
        "Probably the most spectacular view I ever had was driving along the highway no. 1 between LA and SF",
        "He acting all loyal in this pic, but that lil b* bit me the next minute",
        "Probably the most spectacular view I ever had was driving along the highway no. 1 between LA and SF.",
        "Hey @google, Hire me, if you want your bike back! #mountainview #google #siliconvalley",
        "Throwback to an amazing road trip this fall. What a view! Def. worth the hike #hike #nature #travel #roadtrip #yosemite",
        "Keep your eyes on the stars and keep your feet on the ground",
        "You can clearly see I enjoyed being back in SoCal. That place is so beautiful...",
        
         "he Pacific #beers #beach #chilling #california #home #uglyfeet @goldclapmusic good day dude",
        "Never hold back #weekendmood",
        "Thank you @kaleandme for this awesome party! #crewlove @reiseminister_coco",
        "tb to one of my best games ever Still holding the club record for most receptions per game 2013 vs Cottbus W31:15 13 receptions, 193 yards, 3 TDs Thank you @zbreez_ for the trust and the passes !",
        "Throwback to the best winter ever. My kids group. I gave us the name the lil crackheads, because they got me cracking up all the time! #fuckhashtags",
        "Won this one big time 60:20, but it cost us some good men... #hospitalreunion",
        "#Repost @luebeckcougars Much way for improvement though personally. #getbettereveryday Gamestats #cougars #cougarnation #cougarspride #luebeck #luebeckcougars #football #americanfootball #gfl #amgid #afvd #gridiron #awaygame #ritterhude #badgers",
        "100 days credits @gridiron_design_s",
        "New Year's Eve in the alps with the crew. #illuminati",
        "#tb to the good old times when sunday meant gameday and mondays newspaper Sitting in the library studying right now I wish I could go back in time",
        
        "What you believe becomes reality. Create your own world #sundaymood",
        "Mom, can we have one?",
        "Trying to get my surf game up. I think I have a much better form on the beach than in the water though haha #germansaintmermaids @lennartmhr",
        "Table Mountain from Lion's Head @lennartmhr",
         "I hate off-season... ",
         "This shoutout is overdue. Best musical speaker @realcoleworld!",
         "#tb to when my bro @enzeocha_11 was making the Swedish defense look like 8th graders.",
         "Hoffentlich liegt das in der Familie! #crossingfingers #toodamnhot",
         "Touchdown 70:46 crazy game!",
         "Some work by @dani_piedrabuena. Trying to look smart in that suit haha ",
         
         "Back at it",
         "Received this at work. Now my colleagues think I'm famous lol Thank you so much Colin!",
         "#tb to the toughest job I ever had to do good times",
         "office break ",
         "Beach is better. #barcelona #barca #beach #playa #sun #smile #spain #sleepyface"
         "Thank you for such a lovely birthday night.",
         "My new car... I wish. M4 Test Drive. Still having goosebumps #bmw #m4 #madrid #racing",
         "Don't mess with NFL Runningbacks",
         "perfect day at el Retiro",
         "Morning routine. Scooter Ride #madrid #work #watch #scooter #sun #suit"
]

In [3]:
laurents_test = {'name': ['Laurents'], 'type': ['ENFP']}

In [4]:
laurents_row = pd.DataFrame(laurents_test)

In [5]:
laurents_row['text'] = ' '.join(Laurents)

In [6]:
laurents_row

,name,type,text
0,Laurents,ENFP,"Haven't posted in a while, so this random pict..."


In [7]:
def processed(x):
    pattern = r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))'''
    cleaned = re.sub(pattern, ' ', str(x)) 
    cleaned = re.sub(r"[^A-Za-z0-9]+", " ", cleaned)
    cleaned = re.sub(" \d+", " ", cleaned)
    cleaned = cleaned.lower().strip()

    pattern2 = '\w{1,}'
    bag = re.findall(pattern2, cleaned)

    porter = PorterStemmer()
    porter_bag = [porter.stem(word) for word in bag]

    lemmatizer = wordnet.WordNetLemmatizer()
    bag_of_words = [lemmatizer.lemmatize(word) for word in porter_bag]

    stopwords_e = stopwords.words('english')
    bag_of_words = [word for word in bag_of_words if word not in stopwords_e]

    return bag_of_words

In [8]:
text_processed = [processed(i) for i in laurents_row.text]

In [9]:
laurents_row['text_processed'] = text_processed

In [10]:
laurents_row['text_ready'] = laurents_row.text_processed.apply(' '.join)

In [11]:
laurents_row = laurents_row.drop(columns = ['name', 'text', 'text_processed'])

In [12]:
laurents_row

,type,text_ready
0,ENFP,post thi random pictur fill void go within lot...


In [13]:
data = pd.read_csv(r'../data/personalities_cleaned.csv')

In [14]:
data = data.drop(columns = ['Unnamed: 0', 'posts', 'text_processed'])

In [15]:
data.drop(data.index[3559], inplace=True)

In [16]:
data.head()

,type,text_ready
0,INFJ,intj moment sportscent top ten play prank ha l...
1,ENTP,find lack post veri alarm sex bore posit often...
2,INTP,good one cours say know bless cur doe absolut ...
3,INTJ,dear intp enjoy convers day esoter gab natur u...
4,ENTJ,fire anoth silli misconcept approach logic go ...


In [17]:
data = data.append(laurents_row, ignore_index = True)

In [18]:
data2 = data.copy()

In [19]:
# cleaning and processing

data2['I-E'] = data2['type'].astype(str).str[0]
data2['N-S'] = data2['type'].astype(str).str[1]
data2['T-F'] = data2['type'].astype(str).str[2]
data2['J-P'] = data2['type'].astype(str).str[3]

In [20]:
data2.head()

,type,text_ready,I-E,N-S,T-F,J-P
0,INFJ,intj moment sportscent top ten play prank ha l...,I,N,F,J
1,ENTP,find lack post veri alarm sex bore posit often...,E,N,T,P
2,INTP,good one cours say know bless cur doe absolut ...,I,N,T,P
3,INTJ,dear intp enjoy convers day esoter gab natur u...,I,N,T,J
4,ENTJ,fire anoth silli misconcept approach logic go ...,E,N,T,J


In [21]:
data2.tail()

,type,text_ready,I-E,N-S,T-F,J-P
8670,ENFP,thi thread alreadi exist someplac el doe heck ...,E,N,F,P
8671,INTP,mani question thing would take purpl pill pick...,I,N,T,P
8672,INFP,veri conflict right come want child honestli m...,I,N,F,P
8673,INFP,ha long sinc personalitycaf although seem chan...,I,N,F,P
8674,ENFP,post thi random pictur fill void go within lot...,E,N,F,P


In [22]:
x = data2['text_ready']

y_IE = data2['I-E']
y_NS = data2['N-S']
y_TF = data2['T-F']
y_JP = data2['J-P']

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

vector = CountVectorizer(ngram_range=(2, 2)).fit(x) 
X = vector.transform(x)

In [25]:
y_test = list(y_IE[8674])

In [26]:
y_test

['E']

In [27]:
from sklearn.model_selection import train_test_split, cross_val_score

In [29]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=10) # see if can tweak accuracy

from sklearn.ensemble import RandomForestClassifier
rfn = RandomForestClassifier(max_depth=5, random_state=0) # see if can tweak accuracy

from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()



# IE
x_train = X[0:8673]
x_test = X[8674]
y_train = y_IE[0:8673]
y_test = list(y_IE[8674])
knn.fit(x_train, y_train)
ieb_train = knn.score (x_train,y_train)
ieb_test = knn.score (x_train,y_train)
predknn = knn.predict(x_test)
print("I-E RESULTS")
print('Label I-E train score is :',ieb_train)
print('Label I-E test score is :',ieb_test)
print("Confusion Matrix for K-Nearest Neighbors:")
print(confusion_matrix(y_test,predknn))
print("Score:",round(accuracy_score(y_test,predknn)*100,2))
print("Classification Report:",classification_report(y_test,predknn))
print('Prediction is: ', predknn)
print('Actual is: ', y_test)
print("*" * 100)

# NS
x_train = X[0:8673]
x_test = X[8674]
y_train = y_NS[0:8673]
y_test = list(y_NS[8674])
rfn.fit(x_train, y_train)
nsb_train = rfn.score (x_train,y_train)
nsb_test = rfn.score (x_train,y_train)
predrfn = rfn.predict(x_test)
print("N-S RESULTS")
print('Label N-S train score is :',nsb_train)
print('Label N-S test score is :',nsb_test)
print("Confusion Matrix for Random Forests:")
print(confusion_matrix(y_test,predrfn))
print("Score:",round(accuracy_score(y_test,predrfn)*100,2))
print("Classification Report:",classification_report(y_test,predrfn))
print('Prediction is: ', predrfn)
print('Actual is: ', y_test)
print("*" * 100)

# TF
x_train = X[0:8673]
x_test = X[8674]
y_train = y_TF[0:8673]
y_test = list(y_TF[8674])
mnb.fit(x_train, y_train)
tfb_train = mnb.score (x_train,y_train)
tfb_test = mnb.score (x_train,y_train)
predmnb = mnb.predict(x_test)
print("T-F Results")
print('Label T-F train score is :',tfb_train)
print('Label T-F test score is :',tfb_test)
print("Confusion Matrix for Multinomial Naive Bayes:")
print(confusion_matrix(y_test,predmnb))
print("Score:",round(accuracy_score(y_test,predmnb)*100,2))
print("Classification Report:",classification_report(y_test,predmnb))
print('Prediction is: ', predmnb)
print('Actual is: ', y_test)
print("*" * 100)

# JP
x_train = X[0:8673]
x_test = X[8674]
y_train = y_JP[0:8673]
y_test = list(y_JP[8674])
mnb.fit(x_train, y_train)
jpb_train = mnb.score (x_train,y_train)
jpb_test = mnb.score (x_train,y_train)
predmnb = mnb.predict(x_test)
print("J-P Results")
print('Label J-P train score is :',jpb_train)
print('Label J-P test score is :',jpb_test)
print("Confusion Matrix for Multinomial Naive Bayes:")
print(confusion_matrix(y_test,predmnb))
print("Score:",round(accuracy_score(y_test,predmnb)*100,2))
print("Classification Report:",classification_report(y_test,predmnb))
print('Prediction is: ', predmnb)
print('Actual is: ', y_test)
print("*" * 100)

I-E RESULTS
Label I-E train score is : 0.7696298858526461
Label I-E test score is : 0.7696298858526461
Confusion Matrix for K-Nearest Neighbors:
[[0 1]
 [0 0]]
Score: 0.0
Classification Report:               precision    recall  f1-score   support

           E       0.00      0.00      0.00       1.0
           I       0.00      0.00      0.00       0.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0

Prediction is:  ['I']
Actual is:  ['E']
****************************************************************************************************


/usr/local/Cellar/jupyterlab/1.2.4/libexec/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/Cellar/jupyterlab/1.2.4/libexec/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


N-S RESULTS
Label N-S train score is : 0.8619854721549637
Label N-S test score is : 0.8619854721549637
Confusion Matrix for Random Forests:
[[1]]
Score: 100.0
Classification Report:               precision    recall  f1-score   support

           N       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

Prediction is:  ['N']
Actual is:  ['N']
****************************************************************************************************
T-F Results
Label T-F train score is : 1.0
Label T-F test score is : 1.0
Confusion Matrix for Multinomial Naive Bayes:
[[0 1]
 [0 0]]
Score: 0.0
Classification Report:               precision    recall  f1-score   support

           F       0.00      0.00      0.00       1.0
           T       0.00      0.00      0.00       0.0

    accuracy                           0.00       1.0
   macro avg      

/usr/local/Cellar/jupyterlab/1.2.4/libexec/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/Cellar/jupyterlab/1.2.4/libexec/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


J-P Results
Label J-P train score is : 1.0
Label J-P test score is : 1.0
Confusion Matrix for Multinomial Naive Bayes:
[[0 0]
 [1 0]]
Score: 0.0
Classification Report:               precision    recall  f1-score   support

           J       0.00      0.00      0.00       0.0
           P       0.00      0.00      0.00       1.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0

Prediction is:  ['J']
Actual is:  ['P']
****************************************************************************************************


/usr/local/Cellar/jupyterlab/1.2.4/libexec/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/Cellar/jupyterlab/1.2.4/libexec/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
